In [ ]:
import os
from functools import reduce
import json                                                                     
import random
from collections import OrderedDict
import pdb
from copy import deepcopy
from operator import itemgetter
from itertools import islice

from imp import reload

import pycrfsuite                                                               
import pandas as pd                                                             
import numpy as np                    
from IPython.display import Audio

import resulter
reload(resulter)
from resulter import Resulter

In [ ]:
building_name = 'ebu3b'
token_type = 'justseparate'
label_type = 'label'

data_available_buildings = []

with open("metadata/%s_char_sentence_dict_%s.json" \
          % (building_name, token_type), "r") as fp:
    sentence_dict = json.load(fp)
sentence_dict = dict((srcid, [char for char in sentence]) for (srcid, sentence) in sentence_dict.items())
    
with open('metadata/{0}_char_category_dict.json'.format(building_name), 'r') as fp:
    char_category_dict = json.load(fp)
with open('metadata/{0}_char_label_dict.json'.format(building_name), 'r') as fp:
    char_label_dict = json.load(fp)
    
if label_type=='label':
    label_dict = char_label_dict
elif label_Type=='category':
    label_dict = char_category_dict

#sentence_dict = dict()
#for srcid, sentence_label in label_dict.items():
#    sentence_dict[srcid] = list(map(itemgetter(0), sentence_label))
          
    
if building_name in data_available_buildings:
    with open("model/fe_%s.json"%building_name, "r") as fp:
        data_feature_dict = json.load(fp)
    with open("model/fe_%s.json"%building_name, "r") as fp:
        normalized_data_feature_dict = json.load(fp)
    for srcid in sentence_dict.keys():
        if not normalized_data_feature_dict.get(srcid):
            normalized_data_feature_dict[srcid] = None

cluster_filename = 'model/%s_word_clustering_%s.json' % (building_name, token_type)
if os.path.isfile(cluster_filename):
    with open(cluster_filename, 'r') as fp:
        cluster_dict = json.load(fp)

In [ ]:
#def calc_features(sentence,labels):
def calc_features_deprecated(sentence, ts_features=None):
    sentenceFeatures = list()
    for i, word in enumerate(sentence):
        features = [
            'word.lower=' + word.lower(),
            'word.isdigit=%s' % word.isdigit()
        ]
        if i>0:
            features.extend([
                    '-1:word.lower=' + sentence[i-1].lower()
                ])
        else:
            features.append('BOS')
            
        if i<len(sentence)-1:
            features.extend([
                    '+1:word.lower=' + sentence[i+1].lower(),
                ])
        else:
            features.append('EOS')
        sentenceFeatures.append(features)
    return sentenceFeatures

#def calc_features(sentence,labels):
def calc_features(sentence, ts_features=None):
    sentenceFeatures = list()
    sentence = ['$' if c.isdigit() else c for c in sentence]
    for i, word in enumerate(sentence):
        features = {
            'word.lower='+word.lower(): 1.0,
            'word.isdigit': float(word.isdigit())
        }
        if i==0:
            features['BOS'] = 1.0
        else:
            features['-1:word.lower=' + sentence[i-1].lower()] = 1.0
            
        if i in [0,1]:
            features['SECOND'] = 1.0
        else:
            features['-2:word.lower=' + sentence[i-2].lower()] = 1.0
            
        if i<len(sentence)-1:
            features['+1:word.lower='+sentence[i+1].lower()] = 1.0
        else:
            features['EOS'] = 1.0
        if ts_features:
            for j, feat in enumerate(ts_features):
                features['ts_feat_'+str(j)] = feat
        sentenceFeatures.append(features)
    return sentenceFeatures

In [ ]:
# Learning Sample Selection
USE_CLUSTER = True
SAMPLE_NUM = 50

sample_srcid_list = set()
length_counter = lambda x:len(x[1])
ander = lambda x,y:x and y
labeled_srcid_list = list(label_dict.keys())
if USE_CLUSTER:
    sample_cnt = 0
    sorted_cluster_dict = OrderedDict(sorted(cluster_dict.items(),key=length_counter, reverse=True))
    while len(sample_srcid_list) < SAMPLE_NUM:
        for cluster_num, srcid_list in sorted_cluster_dict.items():
            valid_srcid_list = set(srcid_list).intersection(set(labeled_srcid_list))\
                                .difference(set(sample_srcid_list))
            if len(valid_srcid_list) > 0:
                sample_srcid_list.add(random.choice(list(valid_srcid_list)))
            if len(sample_srcid_list) >= SAMPLE_NUM:
                break
else:
    random_idx_list = random.sample(range(0,len(labeled_srcid_list)),SAMPLE_NUM)
    sample_srcid_list = [labeled_srcid_list[i] for i in random_idx_list]
    

In [ ]:
# Cluster counting (remove later)
cluster_counter_dict = dict((cluster_id,0) for cluster_id in cluster_dict.keys())

for srcid in sample_srcid_list:
    for cluster_id, srcid_list in cluster_dict.items():
        if srcid in srcid_list:
            cluster_counter_dict[cluster_id] += 1

In [ ]:
#%%time
trainer = pycrfsuite.Trainer(verbose=False, algorithm='pa')#{‘lbfgs’, ‘l2sgd’, ‘ap’, ‘pa’, ‘arow’}
trainer.set_params({'feature.possible_states': True,
                    'feature.possible_transitions':True,                    
                   })
#for srcid, setence in sentence_dict.items():
for srcid in sample_srcid_list:
    sentence = list(map(itemgetter(0), label_dict[srcid]))
    labels = list(map(itemgetter(1), label_dict[srcid]))
    #trainer.append(pycrfsuite.ItemSequence(calc_features(sentence, labels)), labels)
    if building_name in data_available_buildings:
        data_features = normalized_data_feature_dict[srcid]
    else:
        data_features = None
    trainer.append(pycrfsuite.ItemSequence(
        calc_features(sentence, data_features)), labels)

In [ ]:
trainer.get_params()

In [ ]:
#%%time
crf_model_file = 'model/crf_params_char_{0}_{1}_{2}_{3}_{4}.crfsuite'\
                 .format(building_name, token_type, label_type, str(SAMPLE_NUM), 'clustered' if USE_CLUSTER else 'notclustered')
trainer.train(crf_model_file)

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open(crf_model_file)

In [ ]:
#%%time
predicted_dict = dict()
score_dict = dict()
for srcid, sentence in sentence_dict.items():
    if building_name in data_available_buildings:
        data_features = normalized_data_feature_dict[srcid]
    else:
        data_features = None
    predicted = tagger.tag(calc_features(sentence, data_features))
    predicted_dict[srcid] = predicted
    score_dict[srcid] = tagger.probability(predicted)

In [ ]:
DEBUG = True
precisionOfTrainingDataset = 0.0                                                
totalWordCount = 0.0                                                            
labeledSrcidList = list(label_dict.keys())
randomSrcidList = sample_srcid_list
resulter = Resulter()
result_dict = dict()

for srcid in sentence_dict.keys():
    sentence = sentence_dict[srcid]                                              
    predicted = predicted_dict[srcid]
    if not srcid in list(label_dict.keys()):                                       
        for word, predTag in zip(sentence, predicted): 
            if DEBUG:
                pass
                #print('{:20s} {:20s}'.format(word,predTag))
            else:
                pass
    else:          
        if srcid in randomSrcidList:
            print("===================== %s TRAINING ==================== %e" 
                  % (srcid, score_dict[srcid]))
        else:
            print("===================== %s TEST        ================= %e" 
                  % (srcid, score_dict[srcid]))
        printing_pairs = list()
        sentence_label = label_dict[srcid]
        label_list = list(map(itemgetter(1), sentence_label))
        #sentence = list(map(itemgetter(0), sentence_label))
        if srcid=='514_0_3006463':
            #pdb.set_trace()
            pass
        resulter.add_one_result(srcid, sentence, predicted, label_list)
        
        for word, predTag, origLabel in zip(sentence, predicted, label_list):
            printing_pair = [word,predTag,origLabel]
            if origLabel=='soda':
                pdb.set_trace()
            if predTag==origLabel:                                          
                precisionOfTrainingDataset += 1                             
                printing_pair = ['O'] + printing_pair                       
            else:                                                           
                printing_pair = ['X'] + printing_pair                                       
            totalWordCount += 1                                             
            printing_pairs.append(printing_pair)                                
        if 'X' in [pair[0] for pair in printing_pairs]:                         
            for (flag, word, predTag, origLabel) in printing_pairs:
                if DEBUG:
                    print('{:5s} {:20s} {:20s} {:20s}'\
                          .format(flag, word, predTag, origLabel))                                          

print("# Learning Sample: ",len(randomSrcidList))
print("# Test Sample: ",len(label_dict) - len(randomSrcidList))
print("Precision: ", precisionOfTrainingDataset/totalWordCount)

In [ ]:
resulter.serialize_result('result/total.json')
resulter.summarize_result()
resulter.serialize_summary('result/summary.json')

In [ ]:
len(label_list)

In [ ]:
print("Finished!!!")
sound_file = 'etc/fins_success.wav'
Audio(url=sound_file, autoplay=True)